<center>
</center>

# Watson Speech to Text Translator

Estimated time needed: **25** minutes

## Objectives

After completing this lab you will be able to:

*   Operate a Speech to Text Translator through an API


### Introduction

<p>In this notebook, you will learn to convert an audio file of an English speaker to text using a Speech to Text API. Then, you will translate the English version to a Spanish version using a Language Translator API. <b>Note:</b> You must obtain the API keys and endpoints to complete the lab.</p>


<div class="alert alert-block alert-info" style="margin-top: 20px">
<h2>Table of Contents</h2>
<ul>
    <li><a href="https://#ref0">Speech To Text</a></li>
    <li><a href="https://#ref1">Language Translator</a></li>
    <li><a href="https://#ref2">Exercise</a></li>
</ul>
</div>


In [1]:
#you will need the following library 
!pip install ibm_watson wget

     |████████████████████████████████| 412 kB 15.9 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 68 kB 9.6 MB/s  eta 0:00:01
  Created wheel for ibm-watson: filename=ibm_watson-5.3.0-py3-none-any.whl size=408872 sha256=7404fb9cec778d64c543189a2c48b8347fb90590c78976622163ed8157d0a793
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/79/7c/1e/3fc8483af029cd8cb8c803eeaf282cbeb60724130d9dca4b83
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=79ca3e13884fa7c9f8d4b8d6aeecb5edacfc58590bb6c6b930e5fa11dbe5dab0
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built ibm-watson wget


<h2 id="ref0">Speech to Text</h2>


<p>First we import <code>SpeechToTextV1</code> from <code>ibm_watson</code>. For more information on the API, please click on this <a href="https://cloud.ibm.com/apidocs/speech-to-text?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01&code=python">link</a>.</p>


In [2]:
from ibm_watson import SpeechToTextV1 
import json
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

<p>The service endpoint is based on the location of the service instance, we store the information in the variable URL. To find out which URL to use, view the service credentials and paste the url here.</p>


In [4]:
url_s2t = "https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/906e48df-067b-430d-aaae-63b7aab6d23b"
url_s2t

'https://api.eu-gb.speech-to-text.watson.cloud.ibm.com/instances/906e48df-067b-430d-aaae-63b7aab6d23b'

<p>You require an API key, and you can obtain the key on the <a href="https://cloud.ibm.com/resources?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01">Dashboard </a>.</p>


In [5]:
iam_apikey_s2t = "bujJGSOWr_PHmQhIrgr-mN-apqMHV1Q_oV4b9285s9kh"
iam_apikey_s2t

'bujJGSOWr_PHmQhIrgr-mN-apqMHV1Q_oV4b9285s9kh'

<p>You create a <a href="http://watson-developer-cloud.github.io/python-sdk/v0.25.0/apis/watson_developer_cloud.speech_to_text_v1.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01">Speech To Text Adapter object</a> the parameters are the endpoint and API key.</p>


In [6]:
authenticator = IAMAuthenticator(iam_apikey_s2t)
print(authenticator)
s2t = SpeechToTextV1(authenticator=authenticator)
s2t.set_service_url(url_s2t)
s2t

<p>Lets download the audio file that we will use to convert into text.</p>


In [7]:
!wget -O PolynomialRegressionandPipelines.mp3  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/data/PolynomialRegressionandPipelines.mp3



--2022-01-01 04:24:20--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0101EN-SkillsNetwork/labs/Module%205/data/PolynomialRegressionandPipelines.mp3
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4234179 (4.0M) [audio/mpeg]
Saving to: ‘PolynomialRegressionandPipelines.mp3’

PolynomialRegressio 100%[===================>]   4.04M  1.34MB/s    in 3.0s    

2022-01-01 04:24:24 (1.34 MB/s) - ‘PolynomialRegressionandPipelines.mp3’ saved [4234179/4234179]



<p>We have the path of the .wav file we would like to convert to text</p>


In [8]:
filename='PolynomialRegressionandPipelines.mp3'

<p>We create the file object <code>wav</code> with the wav file using <code>open</code>. We set the <code>mode</code> to  "rb" ,  this is similar to read mode, but it ensures the file is in binary mode. We use the method <code>recognize</code> to return the recognized text. The parameter <code>audio</code> is the file object <code>wav</code>, the parameter <code>content_type</code> is the format of the audio file.</p>


In [10]:
with open(filename, mode="rb")  as wav:
    response = s2t.recognize(audio=wav, content_type='audio/mp3')

<p>The attribute result contains a dictionary that includes the translation:</p>


In [11]:
response.result

{'result_index': 0,
 'results': [{'final': True,
   'alternatives': [{'transcript': 'in this video we will cover polynomial regression and pipelines ',
     'confidence': 0.94}]},
  {'final': True,
   'alternatives': [{'transcript': "what do we do when a linear model is not the best fit for our data let's look into another type of regression model the polynomial regression we transform our data into a polynomial then use linear regression to fit the parameters that we will discuss pipelines pipelines are way to simplify your code ",
     'confidence': 0.9}]},
  {'final': True,
   'alternatives': [{'transcript': "polynomial regression is a special case of the general linear regression this method is beneficial for describing curvilinear relationships what is a curvilinear relationship it's what you get by squaring or setting higher order terms of the predictor variables in the model transforming the data the model can be quadratic which means the predictor variable in the model is squar

In [12]:
from pandas import json_normalize

json_normalize(response.result['results'],"alternatives")

,transcript,confidence
0,in this video we will cover polynomial regress...,0.94
1,what do we do when a linear model is not the b...,0.90
2,polynomial regression is a special case of the...,0.95
3,the model can be cubic which means the predict...,0.95
4,there also exists higher order polynomial regr...,0.91
5,let's look at an example from our data we gene...,0.89
6,in python we do this by using the poly fit fun...,0.92
7,negative one point five five seven X. one cute...,0.90
8,consider the feature shown here applying the m...,0.90
9,pipeline sequentially perform a series of tran...,0.89


In [13]:
response

<p>We can obtain the recognized text and assign it to the variable <code>recognized_text</code>:</p>


In [14]:
recognized_text=response.result['results'][0]["alternatives"][0]["transcript"]
print(recognized_text)
type(recognized_text)

in this video we will cover polynomial regression and pipelines 


str

<h2 id="ref1">Language Translator</h2>


<p>First we import <code>LanguageTranslatorV3</code> from ibm_watson. For more information on the API click <a href="https://cloud.ibm.com/apidocs/speech-to-text?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01&code=python"> here</a></p>


In [15]:
from ibm_watson import LanguageTranslatorV3

<p>The service endpoint is based on the location of the service instance, we store the information in the variable URL. To find out which URL to use, view the service credentials.</p>


In [20]:
url_lt='https://api.us-south.language-translator.watson.cloud.ibm.com/instances/0e4cb466-553e-44a9-8f37-274baf40350d'

<p>You require an API key, and you can obtain the key on the <a href="https://cloud.ibm.com/resources?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01">Dashboard</a>.</p>


In [21]:
apikey_lt='HsqIfYZAl15Xlcu8J7YMRhEeFeFn93tZAEKhxBTL86m8'

<p>API requests require a version parameter that takes a date in the format version=YYYY-MM-DD. This lab describes the current version of Language Translator, 2018-05-01</p>


In [22]:
version_lt='2018-05-01'

<p>we create a  Language Translator object <code>language_translator</code>:</p>


In [23]:
authenticator = IAMAuthenticator(apikey_lt)
language_translator = LanguageTranslatorV3(version=version_lt,authenticator=authenticator)
language_translator.set_service_url(url_lt)
language_translator

<p>We can get a Lists the languages that the service can identify.
The method Returns the language code.  For example English (en) to  Spanis (es) and name of each language.</p>


In [24]:
from pandas import json_normalize

json_normalize(language_translator.list_identifiable_languages().get_result(), "languages")

,language,name
0,af,Afrikaans
1,ar,Arabic
2,az,Azerbaijani
3,ba,Bashkir
4,be,Belarusian
...,...,...
71,uk,Ukrainian
72,ur,Urdu
73,vi,Vietnamese
74,zh,Simplified Chinese


<p>We can use the method <code>translate</code>. This will translate the text. The parameter text is the text, Model_id is the type of model we would like to use use we use list the language. In this case, we set it to 'en-es' or English to Spanish. We get a Detailed Response object translation_response</p>


In [25]:
translation_response = language_translator.translate(\
    text=recognized_text, model_id='en-es')
translation_response

<p>The result is a dictionary.</p>


In [26]:
translation=translation_response.get_result()
translation

{'translations': [{'translation': 'en este vídeo cubriremos la regresión polinómica y las tuberías '}],
 'word_count': 10,
 'character_count': 64}

<p>We can obtain the actual translation as a string as follows:</p>


In [27]:
spanish_translation =translation['translations'][0]['translation']
spanish_translation 

'en este vídeo cubriremos la regresión polinómica y las tuberías '

<p>We can translate back to English</p>


In [28]:
translation_new = language_translator.translate(text=spanish_translation ,model_id='es-en').get_result()

<p>We can obtain the actual translation as a string as follows:</p>


In [29]:
translation_eng=translation_new['translations'][0]['translation']
translation_eng

'in this video we will cover the polynomial regression and the pipes '

<br>


<h2>Quiz</h2>


Translate to French.


In [30]:
# Write your code below and press Shift+Enter to execute
French_translation=language_translator.translate(
    text=translation_eng , model_id='en-fr').get_result()

French_translation['translations'][0]['translation']

'Dans cette vidéo nous couvrons la régression polynomiale et les tuyaux '

<h3>Language Translator</h3>


<b>References</b>


[https://cloud.ibm.com/apidocs/speech-to-text?code=python](https://cloud.ibm.com/apidocs/speech-to-text?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01&code=python)


[https://cloud.ibm.com/apidocs/language-translator?code=python](https://cloud.ibm.com/apidocs/language-translator?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0101ENSkillsNetwork19487395-2021-01-01&code=python)
